In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder ,LabelBinarizer
from sklearn import preprocessing , ensemble
from sklearn.svm import SVR
from math import sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

In [14]:
# 原始資料
'''
columns = ['Age', 'Workclass', 'fnlwgt', 'education'
    , 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'salary']
'''
columns = ['Age','Workclass','fnlgwt','Education'
    ,'EdNum','MaritalStatus','Occupation','Relationship','Race','Sex','CapitalGain','CapitalLoss',
           'HoursPerWeek','Country','Income']


df_train_set = pd.read_csv('datasets/adult.data', names=columns, na_values=' ?')
df_test_set = pd.read_csv('datasets/adult.test', names=columns, skiprows=1, na_values=' ?')

print(df_train_set.shape)
print(df_test_set.shape)

(32561, 15)
(16281, 15)


In [15]:
df_train_set.drop('fnlgwt', axis=1, inplace=True)
df_test_set.drop('fnlgwt', axis=1, inplace=True)
df_train_set = df_train_set.dropna()
df_test_set = df_test_set.dropna()
print(df_train_set.shape)
print(df_test_set.shape)
df_test_set.head(20000)

(30162, 14)
(15060, 14)


,Age,Workclass,Education,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
5,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16275,33,Private,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K.
16276,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16278,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [16]:
df_train_set.drop(["Country", "Education"], axis=1, inplace=True) 
df_test_set.drop(["Country", "Education"], axis=1, inplace=True)
#df_train_set.head(20)

In [17]:
colnames = list(df_train_set.columns)
colnames.remove('Age')
colnames.remove('EdNum')
colnames = ['AgeGroup', 'EduGroup']+colnames

labels = ["{0}-{1}".format(i, i+9) for i in range(0,100,10)]
df_train_set['AgeGroup'] = pd.cut(df_train_set.Age, range(0,101,10), right = False, labels = labels)
df_test_set['AgeGroup'] = pd.cut(df_test_set.Age, range(0,101,10), right = False, labels = labels) 

labels = ["{0}-{1}".format(i,i+4) for i in range(0,20,5)]    
df_train_set['EduGroup'] = pd.cut(df_train_set.EdNum, range(0,21,5), right = False, labels = labels)   
df_test_set['EduGroup'] = pd.cut(df_test_set.EdNum, range(0,21,5), right = False, labels = labels)

df_train_set = df_train_set[colnames]
df_test_set = df_test_set[colnames]


In [18]:
from sklearn_pandas import DataFrameMapper
mapper = DataFrameMapper([('AgeGroup', LabelEncoder()),('EduGroup', LabelEncoder()),
                          ('Workclass', LabelEncoder()),('MaritalStatus', LabelEncoder()),
                          ('Occupation', LabelEncoder()),('Relationship', LabelEncoder()),
                          ('Race', LabelEncoder()),('Sex', LabelEncoder()),
                          ('Income', LabelEncoder())], df_out=True, default=None)

cols = list(df_train_set.columns)
cols.remove('Income')
cols = cols[:-3]+['Income']+cols[-3:]  # 将Income列转移到中间去 對齊

df_train = mapper.fit_transform(df_train_set.copy())
df_train.columns = cols
df_test = mapper.fit_transform(df_test_set.copy())
df_test.columns = cols

train_minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
train_data_minmax = train_minmax.fit_transform(df_train[cols])
test_data_minmax = train_minmax.transform(df_test[cols])

df_train = pd.DataFrame(train_data_minmax,columns =cols)
df_test = pd.DataFrame(test_data_minmax,columns =cols)

cols.remove('HoursPerWeek')

In [19]:
x_train, y_train = df_train[cols].values, df_train['HoursPerWeek'].values
x_test, y_test = df_test[cols].values, df_test['HoursPerWeek'].values

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2,random_state=42)
df_train.head(10)

,AgeGroup,EduGroup,Workclass,MaritalStatus,Occupation,Relationship,Race,Sex,Income,CapitalGain,CapitalLoss,HoursPerWeek
0,0.250,0.333333,0.833333,0.666667,0.000000,0.2,1.0,1.0,0.0,0.021740,0.0,0.397959
1,0.500,0.333333,0.666667,0.333333,0.230769,0.0,1.0,1.0,0.0,0.000000,0.0,0.122449
2,0.250,1.000000,0.333333,0.000000,0.384615,0.2,1.0,1.0,0.0,0.000000,0.0,0.397959
3,0.500,1.000000,0.333333,0.333333,0.384615,0.0,0.5,1.0,0.0,0.000000,0.0,0.397959
4,0.125,0.333333,0.333333,0.333333,0.692308,1.0,0.5,0.0,0.0,0.000000,0.0,0.397959
5,0.250,0.333333,0.333333,0.333333,0.230769,1.0,1.0,0.0,0.0,0.000000,0.0,0.397959
6,0.375,1.000000,0.333333,0.500000,0.538462,0.2,0.5,0.0,0.0,0.000000,0.0,0.153061
7,0.500,1.000000,0.666667,0.333333,0.230769,0.0,1.0,1.0,1.0,0.000000,0.0,0.448980
8,0.250,0.333333,0.333333,0.666667,0.692308,0.2,1.0,0.0,1.0,0.140841,0.0,0.500000
9,0.375,0.333333,0.333333,0.333333,0.230769,0.0,1.0,1.0,1.0,0.051781,0.0,0.397959


In [20]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100

In [21]:
def mape_loss_func(labels,preds):
    mask=labels!=0
    return np.fabs((preds[mask]-labels[mask])/labels[mask]).mean()

In [24]:
svr = SVR(kernel='poly',gamma='auto', C=6,degree=6,tol=0.001,coef0=2)
svr.fit(x_train, y_train)
predict = svr.predict(x_validation)

print("RMSE : %.4g" % metrics.mean_squared_error(y_validation, predict,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_validation, predict))

RMSE : 0.134
MAPE : 0.3351


In [111]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv, scoring_fit='neg_mean_squared_error',#neg_mean_squared_error
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

model = SVR()
param_grid = {
    'kernel' :['poly'],
    'gamma' : ['auto'], 
    'C' : [6,7],
    'degree' : [6,7],
    'tol' : [0.001],
    'coef0' : [0,1]

}

model, pred = algorithm_pipeline(x_train, x_validation, y_train, y_validation, model, 
                                 param_grid, cv=3)

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

y_pred=model.predict(x_validation)
y_true=y_validation
print("RMSE : %.4g" % metrics.mean_squared_error(y_true, y_pred,squared=False))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
0.11197586578787513
{'C': 6, 'coef0': 1, 'degree': 6, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.001}
RMSE : 0.1096


In [ ]:
svr = SVR(kernel='poly',gamma='auto', C=6,degree=6,tol=0.001,coef0=1)
svr.fit(x_train, y_train)
predict = svr.predict(x_test)

print("RMSE : %.4g" % metrics.mean_squared_error(y_test, predict,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_test, predict))